In [2]:
from langchain_unstructured import UnstructuredLoader
loader_local = UnstructuredLoader(
    file_path="nlp.pdf",
    strategy="hi_res",
)
docs_local = []
for doc in loader_local.lazy_load():
    docs_local.append(doc)

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: pikepdf C++ to Python logger bridge initialized
INFO: Reading PDF for file: nlp.pdf ...


In [3]:
len(docs_local)

313

In [4]:
print(docs_local[0].page_content)

1. What is Natural Language Processing (NLP)?


In [5]:
first_page_docs = [doc for doc in docs_local if doc.metadata.get("page_number") == 1]

for doc in first_page_docs:
    print(doc.page_content)

1. What is Natural Language Processing (NLP)?
Answer: Natural Language Processing (NLP) is a field of Artificial Intelligence (AI) concerned with the interactions between computers and human (natural) languages. It focuses on enabling computers to understand, interpret, and generate human language in a way that is both meaningful and useful.
2. Mention any two real-world applications of NLP.
Answer:
• Sentiment Analysis: Determining the emotional tone or attitude expressed in text, used for market research, brand monitoring, etc.
• Chatbots and Conversational AI: Building interactive agents that can engage in conversations with humans, provide customer service, answer questions, and more.
3. Define empirical laws in the context of NLP.
Zipf’s Law: When words are ranked according to their frequencies in a large enough collection of texts and then the frequency is plotted against the rank, the result is a logarithmic curve.
Heap's law states that the number of unique words V in a collect

In [6]:
section_data =[]
section =""
for docs in docs_local:
    if docs.metadata.get("category") == "Title":
        
        section_data.append(section)
        section =""
        section+= docs.page_content + "\n"
        
    else:
        section += docs.page_content + "\n"

       
   

In [7]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

# Initialize
model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

# Suppose section_data is a list of section texts
all_chunks = []
raw_chunks =[]

for i, section in enumerate(section_data):
    if len(section) > 0:
        chunks = text_splitter.split_text(section)
        print(f"Section {i} has {len(chunks)} chunks")
        for j, chunk in enumerate(chunks):
            # Optional: Add metadata like section number
            all_chunks.append(Document(
                page_content=chunk,
                metadata={"section_id": i, "chunk_id": j}
            ))
            raw_chunks.append(chunk)

# Create the vector index
embeddings = model.embed_documents(raw_chunks)

""" vectorindex = FAISS.from_documents(all_chunks, embeddings) """




INFO: Use pytorch device_name: cpu
INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Section 1 has 1 chunks
Section 2 has 1 chunks
Section 3 has 1 chunks
Section 4 has 1 chunks
Section 5 has 1 chunks
Section 6 has 1 chunks
Section 7 has 1 chunks
Section 8 has 6 chunks
Section 9 has 2 chunks
Section 10 has 1 chunks
Section 11 has 1 chunks
Section 12 has 3 chunks
Section 13 has 1 chunks
Section 14 has 1 chunks
Section 15 has 1 chunks
Section 16 has 2 chunks
Section 17 has 3 chunks
Section 18 has 1 chunks
Section 19 has 2 chunks
Section 20 has 1 chunks
Section 21 has 2 chunks
Section 22 has 1 chunks
Section 23 has 2 chunks
Section 24 has 1 chunks
Section 25 has 1 chunks
Section 26 has 5 chunks
Section 27 has 5 chunks
Section 28 has 4 chunks
Section 29 has 2 chunks
Section 30 has 1 chunks
Section 31 has 1 chunks
Section 32 has 1 chunks
Section 33 has 1 chunks
Section 34 has 3 chunks
Section 35 has 1 chunks
Section 36 has 1 chunks
Section 37 has 1 chunks
Section 38 has 1 chunks
Section 39 has 2 chunks
Section 40 has 2 chunks
Section 41 has 2 chunks
Section 42 has 3 chunks
S

' vectorindex = FAISS.from_documents(all_chunks, embeddings) '

In [8]:
from sklearn.cluster import KMeans
num_topics = 10
kmeans = KMeans(n_clusters=num_topics, random_state=42)
labels = kmeans.fit_predict(embeddings)


In [9]:
import os
import dotenv
dotenv.load_dotenv()
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=700,
    timeout=None,
    max_retries=2,
) 

In [10]:
import re
cluster_topic_titles = {}
for cluster_id in set(labels):
    rep_idx = list(labels).index(cluster_id)
    rep_chunk = raw_chunks[rep_idx]

    # Ask LLM to name this topic
    # Updated prompt
    prompt = (
        f"Give a very short and clear title for the following topic content.\n"
        f"Just return the title. No explanations, no quotes, no alternatives, no extra text.\n\n"
        f"{rep_chunk}"
    )
    raw_title = llm.invoke(prompt).content.strip()
    clean_title = re.sub(r'^["“”‘’\'*]*|["“”‘’\'*.:]*$', '', raw_title)  # trim quotes, punctuation
    clean_title = re.sub(r'^(Topic Title|Title)\s*[:\-]\s*', '', clean_title, flags=re.IGNORECASE)
    clean_title = clean_title.split("\n")[0].strip()
    cluster_topic_titles[cluster_id] = clean_title
    
labeled_chunks = []
for i, chunk_text in enumerate(raw_chunks):
    chunk_meta = {
        "section_id": all_chunks[i].metadata["section_id"],
        "chunk_id": all_chunks[i].metadata["chunk_id"],
        "cluster_id": int(labels[i]),
        "topic": cluster_topic_titles[labels[i]]
    }
    labeled_chunks.append({
        "text": chunk_text,
        "embedding": embeddings[i],
        "metadata": chunk_meta
    })

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [11]:
vectorstore = FAISS.from_texts(
    texts=[chunk["text"] for chunk in labeled_chunks],
    embedding=model,
    metadatas=[chunk["metadata"] for chunk in labeled_chunks]
)

# === Done! You can now use vectorstore.as_retriever() ===
retriever = vectorstore.as_retriever(search_kwargs=dict(k=5))

INFO: Loading faiss with AVX2 support.
INFO: Successfully loaded faiss with AVX2 support.
INFO: Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.


In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """You are a helpful assistant. 
Use ONLY the following context to answer the question. 
Do NOT use any prior knowledge. 
If the answer is not in the context, respond with "The answer is not available in the provided context."

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)


In [13]:
query = "what is used to  capture the frequency of individul words in a document?"
result = qa_chain.invoke({"query": query})

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [14]:
print("Answer:", result['result'])

Answer: Tokenization. 

Tokenization breaks down text into individual words or phrases, known as tokens. This process captures the frequency of individual words in a document.


In [15]:
for doc in result["source_documents"]:
    print("Chunk:", doc.page_content)
    print("Section ID:", doc.metadata.get("section_id"))
    print("Chunk ID:", doc.metadata.get("chunk_id"))


Chunk: Example:
Consider a corpus such as a collection of news articles. In many English texts, the word “the” is the most frequent. Suppose “the” occurs 10,000 times; then Zipf’s Law suggests that the second most common word might occur roughly 5,000 times, the third about 3,300 times, and so on. Although real data rarely follow the law perfectly (especially at the very high and low frequency ends), the overall pattern is striking. This regularity has been observed across languages and types of text .
Section ID: 40
Chunk ID: 0
Chunk: Tokenization: This breaks down text into individual words or phrases, known as tokens. This is often the first step in text processing.
Section ID: 27
Chunk ID: 4
Chunk: 17. Describe the process of text pre-processing with suitable examples.
Text cleansing Remove faumbers. symbols, marks Creatirg Document Stemming = Keyword Matrix ioKm) Creating 3 corpus Tokenization Removing stop words
Text preprocessing typically involves the following steps:
• Lowerca

In [45]:
print(result["source_documents"])

[Document(id='a8d5a770-c976-484a-99a9-20a50b086c15', metadata={'section_id': 40, 'chunk_id': 0, 'cluster_id': 5, 'topic': '"Empirical Laws in NLP"'}, page_content='Example:\nConsider a corpus such as a collection of news articles. In many English texts, the word “the” is the most frequent. Suppose “the” occurs 10,000 times; then Zipf’s Law suggests that the second most common word might occur roughly 5,000 times, the third about 3,300 times, and so on. Although real data rarely follow the law perfectly (especially at the very high and low frequency ends), the overall pattern is striking. This regularity has been observed across languages and types of text .'), Document(id='7fad23d3-7d3d-4511-8eda-a0e7558298d2', metadata={'section_id': 27, 'chunk_id': 4, 'cluster_id': 0, 'topic': '"Introduction to Natural Language Processing (NLP)"'}, page_content='Tokenization: This breaks down text into individual words or phrases, known as tokens. This is often the first step in text processing.'), D

In [13]:
# Get all stored documents from FAISS
all_docs = vectorstore.similarity_search("placeholder", k=len(vectorstore.docstore._dict))

# Extract and print all unique topics
topics = set()
for doc in all_docs:
    topic = doc.metadata.get("topic")
    if topic:
        topics.add(topic)


print("Unique Topics:")
for topic in sorted(topics):
    print("-", topic)


Unique Topics:
- Applications of NLP
- Data Retrieval
- Definition
- Empirical Laws in NLP
- Finite-State Methods in Morphology
- Minimum Character Edits
- Natural Language Processing (NLP)
- Role of Smoothing in Language Models
- Text Preprocessing Techniques
- Types of Ambiguity in NLP


In [14]:
def get_chunks_by_topic(vectorstore, topic_query):
    all_docs = vectorstore.similarity_search("placeholder", k=len(vectorstore.docstore._dict))
    
    topic_chunks = []
    for doc in all_docs:
        if doc.metadata.get("topic", "").lower() == topic_query.lower():
            topic_chunks.append(doc.page_content)
    
    return topic_chunks


In [43]:
topic = "Applications of NLP"  # or input("Enter topic: ")
chunks = get_chunks_by_topic(vectorstore, topic)

print(f"\nFound {len(chunks)} chunks for topic '{topic}':\n")
for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i}:\n{chunk}\n")



Found 5 chunks for topic 'Applications of NLP':

Chunk 1:
29. With the help of example, explain the process of POS Tagging

Chunk 2:
28. List any two applications of POS tagging.
Two applications of Part-of-Speech (POS) tagging are:
• Syntactic parsing: POS tags of words in a sentence are needed to determine the correct word combinations.
• Named-entity recognition: POS tagging helps in identifying entities and the relationships between them. Named Entity Recognition (NER) is used in applications like information retrieval and question answering systems.

Chunk 3:
Part-of-speech (POS) tagging is an NLP task that involves assigning a grammatical tag (like noun or verb) to each word in a text. POS tagging is a disambiguation task because words can have more than one possible part-of-speech, so the proper tag must be chosen based on the context. Models like Hidden Markov Models (HMMs), Conditional Random Fields (CRF), and neural networks are used, and the accuracy is measured by comparin

In [44]:
from langchain.prompts import PromptTemplate

template = """
You are an expert educational content designer.

Your task is to help retrieve **realistic educational visuals** from the web for the topic **"{topic}"**.

Instructions:
- You will be given text chunks related to the topic.
- Analyze all chunks holistically.
- Identify 1 to 3 key visualizable concepts.
- Based on the concepts, suggest **1 to 3 visual descriptors** that are suitable for web image retrieval.
- These images will be fetched from sources like **DuckDuckGo**
- Later, a separate model (like BLIP2) will describe the retrieved image and generate audio captions — so your job is just to suggest the most **searchable visual ideas**.
-- If only 1 or 2 are needed, output fewer.

Important Notes:
- Your descriptors must be **web-search friendly**, realistic, and likely to return good visuals.
- Do NOT suggest fictional or AI-specific styles like “a digital painting” or “ultra-detailed 4K illustration”.
- You **can suggest things like graphs, real-world scenes, physical experiments**, etc., if they are commonly found online.
- If you mention a physics diagram or formula chart, clarify that it's **just a reference to what's expected to be found** on the web.

Return format strictly as:
{{
  "image1": "<descriptor 1>",
  "image2": "<descriptor 2>",
  "image3": "<descriptor 3>"
}}

Rules:
- If one image is enough, return only "image1".
- Do not include any narration or explanation — only the descriptors.
- Do not use JSON formatting or code — just follow the shown format.

Content Chunks:
{chunks}
"""

prompt = PromptTemplate.from_template(template)


In [45]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Example descriptor generation shots
examples = [
  {
    "topic": "Photosynthesis",
    "chunks": "Photosynthesis is the process by which green plants use sunlight to make food from carbon dioxide and water. Oxygen is released as a byproduct.",
    "descriptors": [
      "Diagram of photosynthesis in plants",
      "Chloroplast structure and function",
      "Photosynthesis chemical reaction chart"
    ]
  },
  {
    "topic": "Newton's Laws of Motion",
    "chunks": "Newton's three laws describe how objects move and interact with forces. The first law is about inertia, second about force and acceleration, and third about action and reaction.",
    "descriptors": [
      "Illustration of Newton's 3 laws with examples",
      "Force and acceleration graph",
      "Action-reaction force diagram"
    ]
  },
  {
    "topic": "Acids and Bases",
    "chunks": "Acids release H+ ions while bases release OH- ions. They are measured on the pH scale. Neutralization reactions occur when acids and bases combine.",
    "descriptors": [
      "pH scale with common substances",
      "Acid-base titration curve",
      "Neutralization reaction diagram"
    ]
  },
  {
    "topic": "Mitosis",
    "chunks": "Mitosis is the process of cell division in which a single cell divides into two identical daughter cells. It includes stages like prophase, metaphase, anaphase, and telophase.",
    "descriptors": [
      "Mitosis stages under microscope",
      "Cell cycle diagram with mitosis",
      "Mitosis vs meiosis comparison chart"
    ]
  },
  {
    "topic": "Ohm's Law",
    "chunks": "Ohm's Law states that the current through a conductor is directly proportional to voltage and inversely proportional to resistance.",
    "descriptors": [
      "Ohm's law triangle diagram",
      "Current-voltage-resistance graph",
      "Simple circuit showing Ohm's Law"
    ]
  },
  {
    "topic": "Periodic Table",
    "chunks": "The periodic table organizes elements based on atomic number and properties. Groups and periods reveal patterns in reactivity and structure.",
    "descriptors": [
      "Modern periodic table labeled",
      "Group trends in periodic table",
      "Periodic table block diagram"
    ]
  },
  {
    "topic": "DNA Structure",
    "chunks": "DNA is composed of nucleotides forming a double helix. It carries genetic instructions using base pairs A-T and G-C.",
    "descriptors": [
      "DNA double helix 3D model",
      "Base pairing in DNA strands",
      "Nucleotide structure diagram"
    ]
  },
  {
    "topic": "Chemical Bonding",
    "chunks": "Atoms bond to achieve stable electron configurations. Common types include ionic, covalent, and metallic bonding.",
    "descriptors": [
      "Ionic vs covalent bonding diagram",
      "Lewis structure examples",
      "Molecular structure of water"
    ]
  },
  {
    "topic": "Thermodynamics",
    "chunks": "Thermodynamics studies energy transfer. Laws of thermodynamics describe conservation of energy and entropy changes.",
    "descriptors": [
      "Laws of thermodynamics flowchart",
      "Heat engine efficiency diagram",
      "Entropy change vs temperature graph"
    ]
  },
  {
    "topic": "Human Digestive System",
    "chunks": "The digestive system breaks down food into nutrients. Key organs include mouth, stomach, intestines, liver, and pancreas.",
    "descriptors": [
      "Human digestive system labeled diagram",
      "Process of digestion infographic",
      "Enzyme function in digestion chart"
    ]
  }
]


In [46]:

topic = "Applications of NLP"  # or input("Enter topic: ")
chunks = get_chunks_by_topic(vectorstore, topic)
# Create individual prompt template for each example
example_prompt = PromptTemplate.from_template(
    "Topic: {topic}\nChunks: {chunks}\nDescriptors: {descriptors}"
)


descriptor_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    
    suffix=template,
    input_variables=["topic", "chunks"]
)


In [47]:
final_prompt =descriptor_prompt.format(topic=topic, chunks=chunks)
response = llm.predict(final_prompt)

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [48]:
print(response)

{
  "image1": "NLP task flowchart with POS tagging",
  "image2": "Named Entity Recognition (NER) system diagram",
  "image3": "Syntactic parsing tree with POS tags"
}


In [49]:
import json
data = json.loads(response)
for key, value in data.items():
    print(f"{value}")

NLP task flowchart with POS tagging
Named Entity Recognition (NER) system diagram
Syntactic parsing tree with POS tags


In [50]:
from duckduckgo_search import DDGS
import requests
import os
from PIL import Image
from io import BytesIO
import time


os.makedirs("retrieved_images", exist_ok=True)

def try_download(image_url, filepath):
    try:
        res = requests.get(image_url, timeout=5)
        if res.status_code == 200 and 'image' in res.headers.get('Content-Type', ''):
            with open(filepath, 'wb') as f:
                f.write(res.content)
            return True
    except:
        pass
    return False
def is_valid_image(image_bytes, min_width=400, min_height=300, min_size_kb=30):
    try:
        img = Image.open(BytesIO(image_bytes))
        width, height = img.size
        file_size_kb = len(image_bytes) / 1024
        return width >= min_width and height >= min_height and file_size_kb >= min_size_kb
    except:
        return False
ddgs = DDGS()
for key, query in data.items():
    time.sleep(10)  # Be kind to the API and avoid rate limiting
    results = ddgs.images(
        keywords=query,
        region="wt-wt",
        safesearch="off",
        size='Large',
        color="Monochrome",
        type_image=None,
        layout=None,
        license_image=None,
        max_results=3,
    )

    found = False
    for r in results:
        if try_download(r['image'], f"retrieved_images/{key}.jpg") and is_valid_image(requests.get(r['image']).content):
            print(f"✅ Downloaded {key}")
            found = True
            break
    if not found:
        print(f"❌ Failed to download any valid image for {key}")


INFO: response: https://duckduckgo.com/?q=NLP+task+flowchart+with+POS+tagging 200
INFO: response: https://duckduckgo.com/i.js?o=json&q=NLP+task+flowchart+with+POS+tagging&l=wt-wt&vqd=4-42596460151203079325124998905729009584&p=-1&f=%2Csize%3ALarge%2Ccolor%3AMonochrome%2C%2C%2C 200


✅ Downloaded image1


INFO: response: https://duckduckgo.com/?q=Named+Entity+Recognition+%28NER%29+system+diagram 200
INFO: response: https://duckduckgo.com/i.js?o=json&q=Named+Entity+Recognition+%28NER%29+system+diagram&l=wt-wt&vqd=4-208603807436081140965577649325434733836&p=-1&f=%2Csize%3ALarge%2Ccolor%3AMonochrome%2C%2C%2C 200


✅ Downloaded image2


INFO: response: https://duckduckgo.com/?q=Syntactic+parsing+tree+with+POS+tags 200
INFO: response: https://duckduckgo.com/i.js?o=json&q=Syntactic+parsing+tree+with+POS+tags&l=wt-wt&vqd=4-93724389505366926544764790691007409280&p=-1&f=%2Csize%3ALarge%2Ccolor%3AMonochrome%2C%2C%2C 200


✅ Downloaded image3


In [ ]:
prompt_template = """
You are a subject matter expert in "{topic}".

You are shown an image. Describe only what is visually present in the image.
Look at the given image and provide an objective description based on what is visually present".

Only describe what can be seen in the image related to the topic. Do not interpret or assume extra context.

Write clear, concise educational content that describes the visual.

If the image includes a graph, explain what the graph depicts (axes, trend, meaning).
If the image includes formulas or equations, briefly explain what they represent in relation to the topic.

Avoid assumptions beyond what is visible. Focus on accurate visual interpretation that is educationally useful.
"""


In [29]:
from duckduckgo_search import DDGS
results = DDGS().images(
    keywords="butterfly",
    region="wt-wt",
    safesearch="off",
    size=None,
    color="Monochrome",
    type_image=None,
    layout=None,
    license_image=None,
    max_results=1,
)
print(results)


INFO: response: https://duckduckgo.com/?q=butterfly 200
INFO: response: https://duckduckgo.com/i.js?o=json&q=butterfly&l=wt-wt&vqd=4-111611603624617945257795521658240076521&p=-1&f=%2C%2Ccolor%3AMonochrome%2C%2C%2C 200


[{'title': 'Butterfly Life Cycle Printout - EnchantedLearning.com', 'image': 'https://www.enchantedlearning.com/bgifs/Bflylifecyclebw.GIF', 'thumbnail': 'https://tse1.mm.bing.net/th?id=OIP.Jm4Nw5xew6zqisIXvJ72egHaIZ&pid=Api', 'url': 'https://www.enchantedlearning.com/subjects/butterfly/activities/printouts/lifecycle.shtml', 'height': 576, 'width': 508, 'source': 'Bing'}]


In [ ]:
import pyttsx3

engine = pyttsx3.init()
voices = engine.getProperty('voices')

# Select male voice (usually index 0 or try looping to find one)
for voice in voices:
    if 'male' in voice.name.lower():
        engine.setProperty('voice', voice.id)
        break

engine.save_to_file("This is a male voice example.", "male_voice.mp3")
engine.runAndWait()


INFO: Could not import comtypes.gen, trying to create it.
INFO: Created comtypes.gen directory: 'c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\comtypes\gen'
INFO: Writing __init__.py file: 'c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\comtypes\gen\__init__.py'
INFO: Using writeable comtypes cache directory: 'c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\comtypes\gen'
INFO: Could not import comtypes.gen._C866CA3A_32F7_11D2_9602_00C04F8EE628_0_5_4: No module named 'comtypes.gen._C866CA3A_32F7_11D2_9602_00C04F8EE628_0_5_4'
INFO: # Generating comtypes.gen._C866CA3A_32F7_11D2_9602_00C04F8EE628_0_5_4
INFO: # Generating comtypes.gen.SpeechLib
INFO: Could not import comtypes.gen._00020430_0000_0000_C000_000000000046_0_2_0: No module named 'comtypes.gen._00020430_0000_0000_C000_000000000046_0_2_0'
INFO: # Generating comtypes.gen._00020430_0000_0000_C000_000000000046_0_2_0
INFO: # Generating comtypes.gen.stdole


In [ ]:
import requests

MIN_IMAGE_SIZE = 20_000  # Minimum valid image size in bytes (e.g., 20KB)

for result in results:
    try:
        image_url = result["image"]
        response = requests.get(image_url, timeout=10)

        if response.status_code == 200:
            if len(response.content) >= MIN_IMAGE_SIZE:
                with open(f"retrieved_images/{key}.jpg", "wb") as f:
                    f.write(response.content)
                print(f"✅ Downloaded {key} from {image_url}")
                break  # move to next descriptor after success
            else:
                print(f"❌ Too small: {key} ({len(response.content)} bytes)")
        else:
            print(f"❌ Failed {key} with status {response.status_code}")
    except Exception as e:
        print(f"❌ Exception downloading {key}: {e}")
